# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from sklearn.model_selection import train_test_split

from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
This is the iris dataset found in [UCI repository](https://archive.ics.uci.edu/ml/datasets/Iris). It is a classification problem in which we need to classify type/class of Iris plant. It contains 50 samples of each class totaling to 150 samples.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# Get dataset
dataset = Dataset.get_by_name(ws, "iris")

iris = dataset.to_pandas_dataframe()
print("Shape of data: ", str(iris.shape))
iris.head()

Shape of data:  (150, 5)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),labels
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# choose a name for experiment
experiment_name = 'udacity_capstone_project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
udacity_capstone_project,quick-starts-ws-129170,Link to Azure Machine Learning studio,Link to Documentation


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below. <br>
**Reasons behind choosing below configurations:**

- **experiment_timeout_minutes** is 15 as the data is very simple for the algorithm to understand and it does not require many models to run on.
- **max_concurrent_iterations** is 5 to run parallel runs.
- **task** is classification as it is a classification problem.
- **primary_metric** is accuracy as the accuracy will give correct results, as there is no data imbalancing problem.
- **training_data** is 'iris' which has 150 samples that will be used for training and validation.
- **label_column_name** is the iris class as the 'labels' is to be predicted.
- **n_cross_validations** is 5 to ensure that the model does not overfit over certain values.

In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    primary_metric='accuracy',
    training_data=iris,
    label_column_name='labels',
    n_cross_validations=5,
    **automl_settings
)

In [12]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_cd8d6a71-4711-414c-84df-68fa7153efde

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

## Run Details
TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_cd8d6a71-4711-414c-84df-68fa7153efde',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-12-04T14:57:47.368758Z',
 'endTimeUtc': '2020-12-04T15:14:09.69455Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity_capstone_project","subscription_id":"5781be4e-7862-42f9-8ae8-e879c711039b","resource_group":"aml-quickstarts-129170","workspace_name":"quick-starts-ws-129170","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_automl_run, best_automl_model = automl_run.get_output()
print(best_automl_run, "\n\n")
print(best_automl_run.get_metrics(), "\n\n")
print(best_automl_model._final_estimator)

Run(Experiment: udacity_capstone_project,
Id: AutoML_cd8d6a71-4711-414c-84df-68fa7153efde_20,
Type: None,
Status: Completed) 


{'precision_score_weighted': 0.9688917748917749, 'precision_score_micro': 0.9666666666666668, 'AUC_micro': 0.9954444444444445, 'average_precision_score_micro': 0.9912013952936464, 'AUC_macro': 0.9938204342574828, 'f1_score_weighted': 0.9667897435897437, 'AUC_weighted': 0.9932193965305421, 'balanced_accuracy': 0.970955710955711, 'recall_score_micro': 0.9666666666666668, 'norm_macro_recall': 0.9564335664335666, 'precision_score_macro': 0.9665601065601066, 'matthews_correlation': 0.9497062506285552, 'average_precision_score_weighted': 0.9888634143634143, 'recall_score_macro': 0.970955710955711, 'average_precision_score_macro': 0.9891663191624342, 'weighted_accuracy': 0.9633933287287206, 'recall_score_weighted': 0.9666666666666668, 'log_loss': 0.12265212475416594, 'f1_score_micro': 0.9666666666666668, 'accuracy': 0.9666666666666668, 'f1_score_macro': 0.96764956680

In [19]:
#TODO: Save the best model
import joblib
joblib.dump(best_automl_model, 'automl_model.joblib')

# Register
model = best_automl_run.register_model(
    model_name='automl_model', 
    model_path='./outputs/'
)
model

Model(workspace=Workspace.create(name='quick-starts-ws-129170', subscription_id='5781be4e-7862-42f9-8ae8-e879c711039b', resource_group='aml-quickstarts-129170'), name=automl_model, id=automl_model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

**Note: As we got accuracy of Hyperdrive more than AutoML hence we didn't deploy this model.**